# 序列构成的数组

## 2.1 内置序列类型概览

* 容器序列: list tuple collections.deque    可以存放不同类型的数据
* 扁平序列: str bytes bytearray memoryview array.array 只能容纳一种类型

* 可变序列: list、bytearray 、array.array、 collections.deque 和 memoryview
* 不可变序列: tuple、str、bytes

## 2.2 列表推导和生成器表达式

In [2]:
x = 'ABC'
dummy = [ord(x) for x in x]
dummy, x

([65, 66, 67], 'ABC')

> <mark/>Python3中都有自己的局部作用域, 表达式内部变量只在局部起作用, 表达式的上下文里同名变量可以被正常引用

#### 同map和filter 对比

In [5]:
symbols = '$¢£¥€¤'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
beyond_ascii

[162, 163, 165, 8364, 164]

In [7]:
beyond_ascii = list(filter(lambda c: c > 127, map(ord, symbols)))
beyond_ascii

[162, 163, 165, 8364, 164]

### 笛卡尔积

In [8]:
colors = ['black', 'write']
sizes = ['S', 'M', 'L']
tshirts = [(color, size) for color in colors for size in sizes]
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('write', 'S'),
 ('write', 'M'),
 ('write', 'L')]

### 生成式表达式

In [11]:
symbols = '$¢£¥€¤'
tuple(ord(x) for x in symbols)  # 使用生成器表达式初始化

(36, 162, 163, 165, 8364, 164)

In [13]:
import array
array.array('I', (ord(x) for x in symbols))

array('I', [36, 162, 163, 165, 8364, 164])

## 2.3 元组不仅仅是不可变的列表 

In [14]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),   # <1>
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

print('{:15} | {:^9} | {:^9}'.format('', 'lat.', 'long.'))
fmt = '{:15} | {:9.4f} | {:9.4f}'
for name, cc, pop, (latitude, longitude) in metro_areas:  # <2>
    if longitude <= 0:  # <3>
        print(fmt.format(name, latitude, longitude))


                |   lat.    |   long.  
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
Sao Paulo       |  -23.5478 |  -46.6358


### 元组拆包

In [15]:
city, _, country, (lon, lat) = metro_areas[0]
city, country, lon, lat

('Tokyo', 36.933, 35.689722, 139.691667)

### 2.3.4 具名元组

参见第一章的例子

### 2.3.5 作为不可变列表的元组

列表不仅仅是不可变的列表, 它们在方法和属性上仍有很多不同

## 2.4 切片

## 2.5 对序列使用 + 和 *

In [16]:
l = [1, 2, 3]
l * 5

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]

In [17]:
5 * 'abcd'

'abcdabcdabcdabcdabcd'

In [19]:
board1 = [[''] * 3 for i in range(3)]
board1[1][2] = 'X'
board1

[['', '', ''], ['', '', 'X'], ['', '', '']]

In [20]:
board2 = [ [''] * 3 ] * 3
board2[1][2] = "O"
board2

[['', '', 'O'], ['', '', 'O'], ['', '', 'O']]

> <mark/> board2 直接对['']*3 对象进行了复制, 实际是对对象引用的复制。在复制过程中就会暴露其本质 <mark>

## 2.6 序列的增量赋值 